# HLS Point Time Series Demo

## Algorithm
The HLS Point Time Series algorithm is hosted here: https://github.com/MAAP-Project/hls-point-time-series

It is used to extract HLS reflectance values at a set of provided points for any temporal window and MGRS tile. The results are written to a parquet file and the results of many jobs can be queried with a single duckdb query! 

In [1]:
from datetime import datetime, timedelta, UTC

from maap.maap import MAAP

maap = MAAP()

### Submit jobs to DPS

Submit a job for several MGRS tiles and a geopackage file with ~110k points. Each job will identify the points that intersect the MGRS tile, find matching HLS granules using the HLS STAC Geoparquet Archive, then extract raster values for each granule + point combination.

Since the algorithm is querying the HLS STAC Geoparquet Archive instead of the CMR STAC API, you can run this process in parallel without worrying about CMR rate limits!

In [1]:
algo_id = "HLSPointTimeSeriesExtraction"
version = "v0.2"
identifier = "demo-20251218"
jobs = {}
for tile in ['14VLQ', '18WXS', '16WFB', '26WMC', '19VDL']:
    job = maap.submitJob(
        algo_id=algo_id,
        version=version,
        identifier=identifier,
        queue="maap-dps-worker-16gb",
        input_file="s3://maap-ops-workspace/shared/henrydevseed/hls-boreal-sample-points.gpkg",
        start_datetime="2013-01-01T00:00:00Z",
        end_datetime="2025-10-31T23:59:59Z",
        mgrs_tile=tile,
        id_col="sample.id",
        bands="red green blue swir_1 swir_2 nir_narrow Fmask",
    )
    jobs[tile] = job.id


## Query the results using duckdb

Each job produces a single parquet file and since the DPS job output folders share a common prefix, we can query the results of all jobs simultaneously using `duckdb`.

In [7]:
import duckdb

parquet_glob = f"s3://maap-ops-workspace/henrydevseed/dps_output/{algo_id}/{version}/{identifier}/**/*.parquet"

# authenticate duckdb to read from S3
duckdb.sql(
    """
    CREATE OR REPLACE SECRET secret (
        TYPE S3,
        PROVIDER CREDENTIAL_CHAIN
    );
    """
)

df = duckdb.sql(

    f"""
    SELECT "sample.id", count(*) from read_parquet('{parquet_glob}')
    GROUP BY "sample.id"
    """
).fetchdf()

df

,sample.id,count_star()
0,S_121528,1767
1,S_83451,1767
2,S_45927,1767
3,S_97210,1767
4,S_65208,1767
...,...,...
939,S_132097,2348
940,S_94200,2348
941,S_68631,2348
942,S_41720,2348


Each row contains all of the band values from a single HLS granule at a point. This dataframe could be used in a time series modeling workflow.

In [12]:
df = duckdb.sql(
    f"""
    SELECT * from read_parquet('{parquet_glob}') WHERE red > -9999 LIMIT 5
    """
).fetchdf()

df

,red,green,blue,swir_1,swir_2,nir_narrow,Fmask,sample.id,time,item_id,__index_level_0__
0,6474,6489,6642,5026,4052,6502,194,S_121528,2013-10-08 12:57:32.422000-05:00,HLS.L30.T14VLQ.2013281T175732.v2.0,0
1,6128,6032,6608,4232,3310,6067,194,S_30219,2013-10-08 12:57:32.422000-05:00,HLS.L30.T14VLQ.2013281T175732.v2.0,1
2,4467,4383,4548,3737,3054,4711,194,S_146139,2013-10-08 12:57:32.422000-05:00,HLS.L30.T14VLQ.2013281T175732.v2.0,2
3,4019,3724,4292,3417,2827,4641,194,S_103630,2013-10-08 12:57:32.422000-05:00,HLS.L30.T14VLQ.2013281T175732.v2.0,3
4,5112,5053,5464,3875,3195,5230,194,S_135817,2013-10-08 12:57:32.422000-05:00,HLS.L30.T14VLQ.2013281T175732.v2.0,4


duckdb can be used to calculate summaries like number of observations per point per year.

In [10]:
df = duckdb.sql(
    f"""
    SELECT "sample.id", year(time) as year, COUNT(*) as n from read_parquet('{parquet_glob}')
    WHERE red > -9999
    GROUP BY "sample.id", year(time)
    ORDER BY year, "sample.id";
    """
).fetchdf()

df

,sample.id,year,n
0,S_100015,2013,23
1,S_100129,2013,15
2,S_100398,2013,20
3,S_100478,2013,13
4,S_100765,2013,22
...,...,...,...
11500,S_99693,2025,109
11501,S_99788,2025,147
11502,S_99806,2025,103
11503,S_999,2025,258
